In [ ]:
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import cv2

import os, itertools, pickle, sys
from pprint import pprint

from util import *
from putil import *
from config import *
%load_ext autoreload
%aimport util
%aimport putil
%aimport config
%autoreload 1

In [ ]:
root_dir = Reconstructions + 'soda_front/2,5_3fps_20s/'

In [ ]:
sys.path.append(Root + 'OpenSfM/')
from opensfm import dataset
from opensfm import features
data = dataset.DataSet(root_dir)

In [ ]:
def show_img(imgs, figsize=(12, 9)):
    if type(imgs) != list:
        imgs = [imgs]
    plt.figure(figsize=figsize)
    alpha = 1
    alpha_dec = 1 / len(imgs)
    for img in imgs:
        plt.imshow(img, alpha=alpha)
        alpha -= alpha_dec
    plt.axis('off')
    plt.show()

In [ ]:
cv2.DMatch(0, 0, 0)

In [ ]:
def pts_to_kps(pts):
    pts[:, :2] = np.round(features.denormalized_image_coordinates(pts[:, :2], V_w, V_h)).astype(int)
    return [cv2.KeyPoint(x, y, size, angle) for x, y, size, angle in pts]

for image1 in data.images():
    if image1.startswith('A002'):
        matches = data.load_matches(image1)
        pts1, desc1, _ = data.load_features(image1)
        kp1 = pts_to_kps(pts1)
        frame1 = data.image_as_array(image1)
        for image2, matches12 in matches.items():
            if image2.startswith('A005') and len(matches12) > 0:
                print(image1, image2, len(matches12))
                
                pts2, desc2, _ = data.load_features(image2)
                kp2 = pts_to_kps(pts2)
                frame2 = data.image_as_array(image2)
                dmatches = [cv2.DMatch(ix1, ix2, 0) for ix1, ix2 in matches12]
                side_by_side = cv2.drawMatches(
                    frame1, kp1,
                    frame2, kp2,
                    dmatches,
                    None,
                    matchesMask=None,
                    matchColor=(0, 255, 0),
                    singlePointColor=(255, 0, 0),
                    flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS
                )
                show_img(side_by_side, figsize=(24, 18))